In [24]:
#函数库
import time
import gc
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,TensorDataset
import numpy as np
import random
import sys
import copy

In [25]:
def gf(a,b):  #有限域乘法
    #print(a,end='')
    #print(b)
    #b1=b
    b=bin(b)
    lenb=len(b)
    tag=[0]*8
    for i in range(lenb-2):#统计1的数量
        if b[lenb-i-1]=='1':
            tag[8-i-1]=1
        #print(lenb-i-1)
    #print(tag)
    if tag[7]==0:
        sum=0
    else:
        sum=a
    te=a
    for i in range(7):
        temp=[0]*9
        te=te<<1
        bin2=bin(te)
        for j in range(len(bin2)-2):
            temp[9-j-1]=bin2[len(bin2)-j-1]
        #if a==63 and b1==25:#调试纠错用
            #print(te)
        if temp[0]=='1':
            te=te^27
        #if a==63 and b1==25:
            #print(te)
        if tag[6-i]==1:
            sum=sum^te
        #if a==63 and b1==25:
            #print(sum)
    sum=sum%256
    return sum

anum=[i for i in range(256)]
ine=[0,5,10,15,4,9,14,3,8,13,2,7,12,1,6,11]
ine1=[5,10,15,0,9,14,3,4,13,2,7,8,1,6,11,12]
ine2=[10,15,0,5,14,3,4,9,2,7,8,13,6,11,12,1]
ine3=[15,0,5,10,3,4,9,14,7,8,13,2,11,12,1,6]
random.seed(124)
rands=np.zeros([160,256],dtype=np.int32)
rands2=np.zeros([160,256],dtype=np.int32)
for i in range(160):
    rands[i][0:256]=random.sample(anum,256)[0:256]
for i in range(160):
    rands2[i][0:256]=random.sample(anum,256)[0:256]
skey=np.zeros(160,dtype=np.int32)
for i in range(10):
    skey[16*i:16*i+16]=random.sample(anum,16)[0:16]
#print(skey) 
#rand1.sort() 1
ivsrands=np.zeros([4,160,256],dtype=np.int32)
ivsrands2=np.zeros([4,160,256],dtype=np.int32)
for k in range(10):
    for i in range(16):
        for j in range(256):
            ivsrands[0][16*k+i][rands[ine[i]+16*k][j]]=j
            ivsrands[1][16*k+i][rands[ine1[i]+16*k][j]]=j
            ivsrands[2][16*k+i][rands[ine2[i]+16*k][j]]=j
            ivsrands[3][16*k+i][rands[ine3[i]+16*k][j]]=j
for k in range(10):
    for i in range(16):
        for j in range(256):
            ivsrands2[0][16*k+i][rands2[ine[i]+16*k][j]]=j
            ivsrands2[1][16*k+i][rands2[ine1[i]+16*k][j]]=j
            ivsrands2[2][16*k+i][rands2[ine2[i]+16*k][j]]=j
            ivsrands2[3][16*k+i][rands2[ine3[i]+16*k][j]]=j
#print(ivsrands[0,5][rands[9][32]]) #1
#ivsrands[0].sort()

In [26]:
#参数
lr=0.01    #学习率
gamma=0   #动量法
epochs=2200    #代数
bs=8*256      #小批量数

#输入数据集  (net.no,mingw)
order=4
num=160
a=256*num*order
x=torch.zeros([a,1,16*48],dtype=torch.float32)
#赋予序列标签
for i in range(a):
    x[i,0,int(i/(256*160))]=1
#赋予网络号标签
for j in range(order):
    for i in range(int(a/4)):
        x[j*160*256+i,0,256+int(i/256)]=1    
#赋明文
for i in range(a):
    x[i,0,512+i%256]=1
#print(x[256*160*2])  #1
x=x.reshape([a,1,16,48])


#目标数据 （miwen) 
#赋初值
y=torch.zeros([a],dtype=torch.int64)
for i in range(4*num):
    for j in range(256):
        y[i*256+j]=j
#print(y[256*31:256*32]) #1

#ivsbyte2
for j in range(4):
    for i in range(int(a/4)):
        y[j*160*256+i]=ivsrands2[j][int(i/256)][y[j*160*256+i]]
#ivsbyte
for j in range(4):
    for i in range(int(a/4)):
        y[j*160*256+i]=ivsrands[j][int(i/256)][y[j*160*256+i]]
#print(y[256*31:256*32]) #1
#mul
for i in range(int(a/4)):
    y[i]=gf(y[i],2)
    y[160*256+i]=gf(y[160*256+i],3)
#skey
for i in range(int(a/4)):
    y[i]=y[i]^(skey[int(i/256)])
    y[160*256+i]=y[160*256+i]^(skey[int(i/256)])
    y[320*256+i]=y[320*256+i]^(skey[int(i/256)])
    y[480*256+i]=y[480*256+i]^(skey[int(i/256)])
#print(y[(32+160)*256+32])
#print(gf(ivsrands[2][32][ivsrands2[2][32][32]],3)^skey[32]) #1
    
    
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#print(sys.getsizeof(x.storage()))
#print(sys.getsizeof(y.storage()))
dataset=TensorDataset(x,y)
#print(dataset.data[0])
data=DataLoader(dataset
               ,batch_size=bs
               ,drop_last=False)
#for i in data:
#    print(i[0].shape)
finsig=torch.zeros([a,256],dtype=torch.float32)
finsig=finsig.cuda()

In [27]:
#网络构建
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(1,16,3,padding=2)    #卷积
        self.pool1=nn.MaxPool2d(2)  #最大池化
        self.conv2=nn.Conv2d(16,32,3,padding=2)   #卷积
        self.pool2=nn.MaxPool2d(2)            #最大池化
        #self.conv3=nn.Conv2d(32,48,3)   #卷积
        #self.pool3=nn.MaxPool2d(2)            #最大池化
        
        self.linear1=nn.Linear(5*13*32,2024,bias=True)  #两层全连接
        self.linear2=nn.Linear(2024,256,bias=False)
    
    def forward(self,x):
        x=torch.tanh(self.conv1(x))   #前向传播 包括激活函数
        x=self.pool1(x)
        x=torch.tanh(self.conv2(x))
        x=self.pool2(x)
        #x=torch.tanh(self.conv3(x))
        #x=self.pool3(x)
        
        x=x.view(-1,5*13*32)
        x=torch.tanh(self.linear1(x))
        output=F.softmax(self.linear2(x),dim=1)
        return output

In [28]:
#定义损失函数以及优化算法及迭代
def train(net,lr):
    start = time.time()
    #MSE函数
    #criterion=nn.MSELoss()
    criterion=nn.NLLLoss() #定义多分类损失函数
    telr=lr
    #opt=optim.SGD(net.parameters(),lr=telr,momentum=gamma) #小批量优化算法
    opt=optim.Adam(net.parameters(),lr=telr,betas=(0.15,0.15),eps=1e-08,amsgrad=True)
    for epoch in range(epochs):
        num1=0
        for m,n in data:
            x1=m.to(device,non_blocking=True)
            y1=n.to(device,non_blocking=True)
            sigma=net.forward(x1)       #前向传播
            sigma1=torch.log(sigma)    #softmax+log+NLLLOSS=多分类交叉熵函数
            loss=criterion(sigma1,y1)   #计算损失函数值
            loss.backward()            #反向传播
            opt.step()                 #权重更新
            opt.zero_grad()            #梯度清零
            #telr=lr-lr*(1-(torch.pow(torch.tanh(torch.tensor(-10+10*(epoch/epochs))),2)))+0.1  #学习率衰减函数
           
            if epoch%50==0 and num1==0:
                print('epoch:{} 该epochs中首个loss值:{} '.format(epoch,loss))
            
            if epoch==epochs-1:
                    finsig[num1*bs:num1*bs+bs]=sigma[0:bs]
            del sigma,sigma1,loss,x1,y1
            gc.collect()
            torch.cuda.empty_cache()
            num1=num1+1
    #统计结果正确的标签数量
    num=0
    fina=torch.max(finsig,1)[1]  #每个长度为256的标签中的最大值所在的索引就是目标字节值，这是训练结果的值
    #y1=torch.max(y,1)[1]        #目标值
    for i in range(a):       
        temp=fina[i]-y[i]
        if temp==0:
            num=num+1
    print('映射成功的个数:%d'%num)
    end = time.time()
    print('程序执行时间: ',end - start)
    return finsig

In [29]:
#模型实例化和确定随机数种子
torch.manual_seed(126)
#torch.cuda.manual_seed(126)
torch.cuda.manual_seed_all(126)
torch.backends.cudnn.deterministic=True
net=Model()
net=net.cuda()

In [30]:
#训练与评估
sigma=train(net,lr)

epoch:0 该epochs中首个loss值:5.546083927154541 
epoch:50 该epochs中首个loss值:5.551842212677002 
epoch:100 该epochs中首个loss值:5.509463310241699 
epoch:150 该epochs中首个loss值:5.295712947845459 
epoch:200 该epochs中首个loss值:4.514838218688965 
epoch:250 该epochs中首个loss值:3.3838934898376465 
epoch:300 该epochs中首个loss值:2.3977091312408447 
epoch:350 该epochs中首个loss值:1.6745210886001587 
epoch:400 该epochs中首个loss值:1.175915002822876 
epoch:450 该epochs中首个loss值:0.8617423176765442 
epoch:500 该epochs中首个loss值:0.6452435851097107 
epoch:550 该epochs中首个loss值:0.501027524471283 
epoch:600 该epochs中首个loss值:0.3955385386943817 
epoch:650 该epochs中首个loss值:0.3177220821380615 
epoch:700 该epochs中首个loss值:0.26178741455078125 
epoch:750 该epochs中首个loss值:0.21835239231586456 
epoch:800 该epochs中首个loss值:0.18420812487602234 
epoch:850 该epochs中首个loss值:0.15711340308189392 
epoch:900 该epochs中首个loss值:0.13608814775943756 
epoch:950 该epochs中首个loss值:0.11921349167823792 
epoch:1000 该epochs中首个loss值:0.10555984079837799 
epoch:1050 该epochs中首个loss值:0.0946510

In [31]:
#求出上述网络中误差最大的标签的值
temp=[0 for i in range(a)]
temp=torch.tensor(temp,dtype=torch.float32)
for i in range(a):
    temp[i]=torch.max(sigma[i],dim=0)[0]
torch.min(temp,dim=0)  #value就是误差最大的值，ind就是对应值的索引号

torch.return_types.min(
values=tensor(0.4025, grad_fn=<MinBackward0>),
indices=tensor(94150))

In [36]:
#保存模型
torch.save(net,'net2v2.1.pkl')

In [64]:
gf(ivsrands[1][0x84],2)

57